In [3]:
###TRANSFORMING SURVERY/PROBLEM SOLUTION

import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from datetime import datetime
import logging
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler
import logging

# Initialize Cloud Logging
# client = google.cloud.logging.Client()
# handler = CloudLoggingHandler(client)
# cloud_logger = logging.getLogger('cloudLogger')
# cloud_logger.setLevel(logging.DEBUG)
# cloud_logger.addHandler(handler)

#### FOR LOCAL TEST######
cloud_logger = logging.getLogger('cloudLogger')
cloud_logger.setLevel(logging.DEBUG)  # Set the logging level to DEBUG

# Create a console handler and set its log level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)  # Set the console handler level to DEBUG

# Define the log format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the console handler to the logger
cloud_logger.addHandler(console_handler)

#### FOR LOCAL TEST######
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import pandas as pd
from datetime import datetime

def format_date(date_str):
    # Check if it's NaN or float first
    if pd.isna(date_str) or isinstance(date_str, float):
        return None  # Return None for invalid dates
    
    # Convert to string and strip any whitespace
    date_str = str(date_str).strip()
    
    # Try to parse the date assuming it's in the format '22/4/2024 15:43'
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
        formatted_date = date_obj.strftime('%d/%m/%Y')
        return formatted_date
    except ValueError:
        pass
    
    # Try to parse the date assuming it's in the format '22/04/24'
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:
        pass
    
    # Try to parse the date assuming it's in the format '22/04/2024'
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:
        pass

    # If all parsing attempts fail, return None
    return None


def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

def process_survey_data(product, source, file_path):

    try:
        data = pd.read_csv(file_path)
        cloud_logger.info("Data loaded successfully.")
    except Exception as e:
        cloud_logger.error(f"Error loading data: {e}")
        raise

    date_column = None
    for col in data.columns:
        if 'date' in col.lower() or data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
            date_column = col
            data[col] = data[col].apply(format_date)
            cloud_logger.info(f"Date column identified: {col}")
            break

    if not date_column:
        cloud_logger.warning("No date column identified.")

    question_texts = data.iloc[0]

    columns_to_drop = []
    for col in data.columns:
        if not (col.startswith('Q') and col[1:].isdigit()):
            if col != date_column:
                columns_to_drop.append(col)
        else:
            if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
                columns_to_drop.append(col)
            elif pd.to_numeric(data[col][1:], errors='coerce').notna().all():
                columns_to_drop.append(col)
            elif data[col][1:].str.strip().str.lower().isin(['yes', 'no']).all():
                columns_to_drop.append(col)

    data.drop(columns=columns_to_drop, inplace=True)
    # cloud_logger.info(f"Dropped columns: {columns_to_drop}")

    data = data[1:]

    data = data[~data.apply(lambda row: row.astype(str).str.contains('QID').any(), axis=1)]
    # cloud_logger.info("Filtered out rows containing 'QID'.")


    for col in data.columns:
        data[col] = data[col].apply(lambda x: x if is_valid_feedback(x) and is_english(x) else None)
        # cloud_logger.info(f"Processed feedback for column: {col}")

    for col in data.columns:
        if col.startswith('Q'):
            question_text = question_texts[col]
            data[col] = data[col].apply(lambda x: f"{source}: {question_text}: {x}" if pd.notna(x) else x)
            # cloud_logger.info(f"Appended question text to column: {col}")

    data.dropna(axis=1, how='all', inplace=True)
    # cloud_logger.info("Dropped columns with all NaN values.")

    long_format_data = data.melt(var_name='Question Code', value_name='Feedback')
    long_format_data = long_format_data.dropna(subset=['Feedback'])
    # cloud_logger.info("Converted data to long format and filtered out rows with NaN feedback.")

    if product != "Others":
        long_format_data['Product'] = product
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        #TODO: Classification
        # long_format_data=classification_defined_products(long_format_data)


    
    else:
        long_format_data['Product'] = None
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        #TODO: Classification
        # long_format_data=classification_undefined_products(long_format_data)

    if date_column:
        long_format_data['Date'] = data[date_column].values

    desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']
    long_format_data = long_format_data.reindex(columns=desired_columns)

    return long_format_data

product= 'Car Loan'
source= 'Product Survey'
path= '/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car Loan__Product Survey__new data.csv'
output_file_path = f'/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car_Loan_result.csv'
data= process_survey_data(product, source, path)
data.to_csv(output_file_path, index=False)
print(f"Data transformation complete. File saved to: {output_file_path}")

    #TODO: Add to SQL Analytics

    # output_file_path = f'/path/to/output/Transformed_{product}_{source}.csv'
    # try:
    #     long_format_data.to_csv(output_file_path, index=False)
    #     cloud_logger.info(f"Data transformation complete. File saved to: {output_file_path}")
    # except Exception as e:
    #     cloud_logger.error(f"Error saving transformed data: {e}")
    #     raise
        

2024-07-21 21:05:28,606 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 21:05:28,610 - cloudLogger - INFO - Date column identified: RecordedDate


KeyError: 'RecordedDate'

In [5]:
import pandas as pd
from datetime import datetime
import logging
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
import re

# Example cloud logger setup
cloud_logger = logging.getLogger('cloudLogger')
cloud_logger.setLevel(logging.DEBUG)  # Set the logging level to DEBUG

# Create a console handler and set its log level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)  # Set the console handler level to DEBUG

# Define the log format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the console handler to the logger
cloud_logger.addHandler(console_handler)

def format_date(date_str):
    if pd.isna(date_str) or isinstance(date_str, float):
        return None
    date_str = str(date_str).strip()
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
        return date_obj.strftime('%d/%m/%Y')
    except ValueError:
        pass
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        return date_obj.strftime('%d/%m/%y')
    except ValueError:
        pass
    try:
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj.strftime('%d/%m/%y')
    except ValueError:
        pass
    return None

def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

def process_survey_data(product, source, file_path):
    try:
        data = pd.read_csv(file_path)
        cloud_logger.info("Data loaded successfully.")
    except Exception as e:
        cloud_logger.error(f"Error loading data: {e}")
        raise

    date_column = None
    for col in data.columns:
        if 'date' in col.lower() or data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
            date_column = col
            data[col] = data[col].apply(format_date)
            cloud_logger.info(f"Date column identified: {col}")
            break

    if not date_column:
        cloud_logger.warning("No date column identified.")

    question_texts = data.iloc[0]

    columns_to_drop = []
    for col in data.columns:
        if not (re.match(r'Q\d+[a-zA-Z]*$', col)):
            if col != date_column:
                columns_to_drop.append(col)
        else:
            if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
                columns_to_drop.append(col)
            elif pd.to_numeric(data[col][1:], errors='coerce').notna().all():
                columns_to_drop.append(col)
            elif data[col][1:].str.strip().str.lower().isin(['yes', 'no']).all():
                columns_to_drop.append(col)

    # Ensure the date column is not dropped
    # if date_column in columns_to_drop:
    #     columns_to_drop.remove(date_column)

    data.drop(columns=columns_to_drop, inplace=True)
    cloud_logger.info(f"Dropped columns: {columns_to_drop}")

    # Keep a copy of the date column before further processing
    if date_column:
        date_series = data[date_column].copy()

    data = data[1:]

    data = data[~data.apply(lambda row: row.astype(str).str.contains('QID').any(), axis=1)]

    for col in data.columns:
        data[col] = data[col].apply(lambda x: x if is_valid_feedback(x) and is_english(x) else None)

    for col in data.columns:
        if col.startswith('Q'):
            question_text = question_texts[col]
            data[col] = data[col].apply(lambda x: f"{source}: {question_text}: {x}" if pd.notna(x) else x)

    data.dropna(axis=1, how='all', inplace=True)

    # Add the date column back to the data before melting
    if date_column:
        data['Date'] = date_series

    long_format_data = data.melt(id_vars=['Date'], var_name='Question Code', value_name='Feedback')
    long_format_data = long_format_data.dropna(subset=['Feedback'])

    if product != "Others":
        long_format_data['Product'] = product
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        # TODO: Classification
        # long_format_data = classification_defined_products(long_format_data)
    else:
        long_format_data['Product'] = None
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        # TODO: Classification
        # long_format_data = classification_undefined_products(long_format_data)

    desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']
    long_format_data = long_format_data.reindex(columns=desired_columns, fill_value=None)

    return long_format_data

# Example usage
product = 'Car Loan'
source = 'Product Survey'
path = '/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car Loan__Product Survey__new data.csv'
output_file_path = f'/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car_Loan_result.csv'
data = process_survey_data(product, source, path)
data.to_csv(output_file_path, index=False)
print(f"Data transformation complete. File saved to: {output_file_path}")


2024-07-21 22:17:37,024 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:17:37,024 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:17:37,024 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:17:37,024 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:17:37,024 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:17:37,027 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:17:37,027 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:17:37,027 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:17:37,027 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:17:37,027 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:17:37,032 - cloudLogger - INFO - Dropped columns: ['Q19_NPS_GROUP']
2024-07-21 22:17:37,032 - cloudLogger - INFO - Dropped columns: ['Q19_NPS_GROUP']
2024-07-21 22:17:37,032 - cloudLogger - INFO - Drop

Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/Test/Car_Loan_result.csv


In [12]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


###TRANSFORMING SURVERY/PROBLEM SOLUTION
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from datetime import datetime
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler

import logging
import re

def format_date(date_str):
    # Check if it's NaN or float first
    if pd.isna(date_str) or isinstance(date_str, float):
        return None  # Return None for invalid dates
    
    # Convert to string and strip any whitespace
    date_str = str(date_str).strip()
    
    # Define the possible input formats
    input_formats = [
        '%d/%m/%Y %H:%M',  # '22/04/2024 15:45'
        '%d/%m/%Y',        # '22/04/2024'
        '%d/%m/%y',        # '22/04/24'
        '%d/%m/%y %H:%M',  # '22/4/24 15:45'
        '%d/%m/%Y'         # '22/4/2024'
    ]
    
    # Try to parse the date with the given formats
    for fmt in input_formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            # Convert to the desired format '22/04/24'
            formatted_date = date_obj.strftime('%d/%m/%y')
            return formatted_date
        except ValueError:
            pass

    # If all parsing attempts fail, return None
    return None

def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False
    
# Initialize Cloud Logging
# client = google.cloud.logging.Client()
# handler = CloudLoggingHandler(client)
# cloud_logger = logging.getLogger('cloudLogger')
# cloud_logger.setLevel(logging.DEBUG)
# cloud_logger.addHandler(handler)

#### FOR LOCAL TEST######
cloud_logger = logging.getLogger('cloudLogger')
cloud_logger.setLevel(logging.DEBUG)  # Set the logging level to DEBUG

# Create a console handler and set its log level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)  # Set the console handler level to DEBUG

# Define the log format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

# Add the console handler to the logger
cloud_logger.addHandler(console_handler)

#### FOR LOCAL TEST######


def process_survey_data(product, source, file_path):

    try:
        data = pd.read_csv(file_path)
        cloud_logger.info("Data loaded successfully.")
    except Exception as e:
        cloud_logger.error(f"Error loading data: {e}")
        raise

    date_column = None
    for col in data.columns:
        if 'date' in col.lower() or data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
            date_column = col
            cloud_logger.info(f"Date column identified: {col}")
            data[col] = data[col].apply(format_date)
            break

    if not date_column:
        cloud_logger.warning("No date column identified.")

    question_texts = data.iloc[0]

    columns_to_drop = []
    for col in data.columns:
        if not (re.match(r'Q\d+[a-zA-Z]*$', col)):
            if col != date_column:
                columns_to_drop.append(col)
        else:
            if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
                columns_to_drop.append(col)
            elif pd.to_numeric(data[col][3:], errors='coerce').notna().all():
                columns_to_drop.append(col)
            elif data[col][3:].str.strip().str.lower().isin(['yes', 'no']).all():
                columns_to_drop.append(col)

    data.drop(columns=columns_to_drop, inplace=True)
    cloud_logger.info(f"Dropped columns: {columns_to_drop}")

    # Ensure date column is copied before further filtering
    if date_column:
        date_series = data[date_column].copy()
        cloud_logger.info(f"Date column copied: {date_series}")

    data = data[1:]

    data = data[~data.apply(lambda row: row.astype(str).str.contains('QID').any(), axis=1)]

    for col in data.columns:
        data[col] = data[col].apply(lambda x: x if is_valid_feedback(x) and is_english(x) else None)

    for col in data.columns:
        if col.startswith('Q'):
            question_text = question_texts[col]
            data[col] = data[col].apply(lambda x: f"{product}: {question_text}: {x}" if pd.notna(x) else x)

    data.dropna(axis=1, how='all', inplace=True)

    # Ensure date column is added back to the data before melting
    if date_column:
        data['Date'] = date_series
        cloud_logger.info(f"Date column added back to data: {data['Date']}")

    long_format_data = data.melt(id_vars=['Date'], var_name='Question Code', value_name='Feedback')
    long_format_data = long_format_data.dropna(subset=['Feedback'])
    cloud_logger.info(f"Converted data to long format and filtered out rows with NaN feedback.")

    if product != "Others":
        long_format_data['Product'] = product
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        #TODO: Classification
        # long_format_data = classification_defined_products(long_format_data)
    else:
        long_format_data['Product'] = None
        long_format_data['Subcategory'] = None
        long_format_data['Feedback Category'] = ''
        long_format_data['Sentiment'] = None
        long_format_data['Sentiment Score'] = None
        long_format_data['Source'] = source

        #TODO: Classification
        # long_format_data = classification_undefined_products(long_format_data)

    desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']
    long_format_data = long_format_data.reindex(columns=desired_columns)

    return long_format_data

product = 'Car Loan'
source = 'Product Survey'
path = '/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car Loan__Product Survey__new data.csv'
output_file_path = f'/Users/phonavitra/Desktop/term 5/Service Studio/Test/Car_Loan_result.csv'
data = process_survey_data(product, source, path)
data.to_csv(output_file_path, index=False)
print(f"Data transformation complete. File saved to: {output_file_path}")


2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,403 - cloudLogger - INFO - Data loaded successfully.
2024-07-21 22:27:27,408 - cloudLogger - INFO - Date column identified: RecordedDate
2024-07-21 22:27:27,408 - cloudLogger - 

Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/Test/Car_Loan_result.csv
